In [30]:
# imports
import pandas as pd
import tiktoken
import os

from openai.embeddings_utils import get_embedding

In [31]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [32]:
# load & inspect dataset
input_datapath = "Reviews.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

/var/folders/pv/8p1pvchj201bllw6r0x2t0p40000gn/T/ipykernel_2593/1307181815.py:3: DtypeWarning: Columns (1,2,3,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_datapath, index_col=0)


,Time,ProductId,UserId,Score,Summary,Text,combined
Id,,,,,,,
1.0,1.303862e+09,B001E4KFG0,A3SGXH7AUHU8GW,5.0,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...
2.0,1.346976e+09,B00813GRG4,A1D87F6ZCVE5NK,1.0,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...


In [33]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 100
df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

100

In [35]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
df.to_csv("fine_food_reviews_with_embeddings_100.csv")